# ゼロから学ぶディープラーニング　8章


解説編<br>
・PyTorchのsequence系データ型について(sequence.pyの解説)<br>
実習編<br>
・100本ノック第9章80のプログラム80.pyの穴埋め<br>
・100本ノック第9章81～83をまとめたプログラム83.pyの穴埋め<br>

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/kobo2020
AttentionSeq2seq.pkl  GoogleNews-vectors-negative300.bin
attention_seq2seq.py  negative_sampling_layer.py
BetterRnnlm.pkl       neko.txt
better_rnnlm.py       nlp100_chap9
cbow_params.pkl       nlp100_chap9.pkl
cbow.py               nlp100_chap9.txt
ch04/                 __pycache__/
ch06/                 PyTorchTutorial1.ipynb
ch07/                 repo10.ipynb
ch07＿pra.ipynb       repo10test.ipynb
ch08/                 repo6.ipynb
ch08_pra.ipynb        repo7.ipynb
common/               repo8.ipynb
data70/               repo9.ipynb
dataset/              simple_rnnlm.py


In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print(device)

cuda:0


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pack_padded_sequence, pad_sequence
# nn.LSTMはtensorとPackedSequence型のデータの両方の形式を受け取ることができる．

# (1) tensorを入力すると，すべての系列について，最後まで入力した結果が得られる
a = torch.ones(3, 5)
b = torch.ones(3, 5) * 2
print("a, b", a, b)

x = torch.stack([a,b])#新たな次元方向に結合
print("x", x)

rnn = nn.LSTM(5, 6, batch_first=True)
print(x)
out, (h, c) = rnn(x)
print("各入力ごとの隠れ状態の出力", out)
print("最後の隠れ状態の出力", h)
print("最後の記憶セル", c)

# nn.LSTMはバッチを入力する場合，系列長が揃っている必要がある．そのため，系列長が異なるデータをバッチ化するため，長い系列に合わせて短い系列をゼロ埋め(padding)する．
# しかし，最大系列長と最小系列長が大きく異なるバッチを入力した場合，最小系列長のデータはゼロベクトルを延々と入力した結果が返ってきてしまうため，適切な結果が得られない．
# そのため，系列長に応じ，各系列の最後の要素をLSTMに入力した時点の出力を得ることが必要である．

# (2) PackedSequenceを入力すると，系列長を考慮した出力が得られる．
a = torch.ones(4, 5) * 1
b = torch.ones(2, 5) * 2
c = torch.ones(1, 5) * 3
print("a, b, cの中身を表示", a, b, c)

# pack_sequenceは異なる長さの系列を1つにまとめたテンソルと，各系列の長さを格納したbatch_sizesテンソルを持つPackedSequence型のデータに変換
packed_inputs = pack_sequence([a, b, c], enforce_sorted=False)
print("packed_inputs", packed_inputs)
print("LSTMへの入力",  packed_inputs)
out, (h, cell) = rnn(packed_inputs)
print("各入力ごとの隠れ状態の出力", out)
print("系列長に応じた最後の隠れ状態の出力", h)
print("系列長に応じた最後の記憶セル", cell)



# RNNに1回入力するだけなら上記でOKだが，自作のDatasetを使用する場合，__getitem__で毎回同じ形状のtensorを返す必要があるのでこれでは都合が悪い．

# pad_sequenceは系列のうち，最も長い系列に合わせ，ゼロ埋め(padding)したtensorを返す．これで各要素の形状が同一となる
pad_inputs = pad_sequence([a, b, c], batch_first=True)
print("pad_inputs", pad_inputs)

# 各系列の長さを計算
seq_sizes = [len(x) for x in [a, b, c]]
print("seq_sizes", seq_sizes)
# paddingした系列と系列長をpackして，PackedSequence型のデータを作成(上のpacked_inputsと同じものができる)
x = pack_padded_sequence(pad_inputs, seq_sizes, batch_first=True, enforce_sorted=False)
print("LSTMへの入力", x)
out, (h, cell) = rnn(x)
print("各入力ごとの隠れ状態の出力", out)
print("系列長に応じた最後の隠れ状態の出力", h)
print("系列長に応じた最後の記憶セル", cell)

# 次の層への入力はhを使用すればO

a, b tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]) tensor([[2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.],
        [2., 2., 2., 2., 2.]])
x tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]])
tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.],
         [2., 2., 2., 2., 2.]]])
各入力ごとの隠れ状態の出力 tensor([[[-0.1961,  0.0627,  0.1872, -0.2197, -0.0727,  0.0408],
         [-0.2531,  0.0860,  0.2769, -0.2992, -0.1036,  0.0582],
         [-0.2724,  0.0974,  0.3235, -0.3296, -0.1176,  0.0651]],

        [[-0.1395,  0.0774,  0.1676, -0.3138, -0.0834,  0.0200],
         [-0.1660,  0.0956,  0.2539, -0.4149, -0.1251,  0.0296],
         [-0.1729,  0.1049,  0.3068, -0.4490, -0.1462,  0.0345]]],
       grad_fn=<Transpo

In [ ]:
import re
from collections import Counter
import pickle

# 単語の最小出現回数
min_freq = 2

train_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/train.txt'#todo ファイル出力ディレクトリを指定
word2id_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/nlp100_chap9'#todo 出力先ファイルパス


# タイトルに含まれる単語をすべてリストword_listに格納
word_list = []
with open(train_path) as f:
    for line in f:
        if not line:
            continue
        data = line.split('\t')

        title = data[1]#todo タイトルを取得
        for word in title.split():
            word_list.append(word)


word_count = dict(Counter(word_list)).items()#todo Counterを使ってword_listの各単語の出現回数をカウント
# 単語をIDに変換するためのdict．keyを単語，valueをIDにする．
word2id = {}


id = 1
for word, freq in word_count:
  
    if freq < min_freq:#todo その単語の出現頻度がmin_freqよりも小さい場合:
        word2id[word] = 0#todo 単語のIDは0とする
    else:
        word2id[word] = id#todo 単語にIDを割り当てる
        #todo IDをインクリメント
        id+= 1


# 表示
print("word_num", len(word2id))
# 保存
pickle.dump(word2id, open(word2id_path, "wb"))#バイナリモードでファイルを保存


word_num 23620


In [ ]:
#Counterとfor文のテスト

testlist = ["apple","greap","apple","orange","apple","orange","apple"]
dic = {}
word_counter = dict(Counter(testlist)).items()
print(dict(word_counter).items())

for word, freq in word_counter:
  print(word)
  print(freq)

dict_items([('apple', 4), ('greap', 1), ('orange', 2)])
apple
4
greap
1
orange
2


In [13]:
#83
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
word2id_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/nlp100_chap9'#todo 80.pyで保存したファイルパスを指定
import numpy as np
word2id = pickle.load(open(word2id_path, "rb"))
#print(word2id)#IDと単語の辞書列型 {単語:ID}

# word_listの単語リストをIDリストに変換する関数
def get_ids(word_list):
    ids = []
    for word in word_list:
        if word in word2id:#todo wordがword2idのキーに含まれる場合，
            ids.append(word2id[word])#todo そのIDをidsにappend
        else:
            ids.append(0)#todo wordがword2idのキーに含まれない場合，0をidsにappend
    return ids

def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    ids_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()#改行を削除
            line = line.split('\t')#todo lineからタイトルを取得
            title = line[1]
            word_list =  title.split(" ")#todo titleを単語に分割
            ids = get_ids(word_list)
            ids = np.array(ids)
            ids_tensor = torch.from_numpy(ids)#todo idsをtensorに変換
            ids_list.append(ids_tensor)#todo ids_tensorをids_listにappend


            category = line[0]#todo lineからカテゴリーを取得
            label = category2num[line[0]]#todo category2numでカテゴリーをラベルに変換
            label_list.append(label)#todo labelをlabel_listにappend

    labels = torch.from_numpy(np.array(label_list))#todo label_listをtensorに変換
    return ids_list, labels



#分類器
class ClassifierRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(ClassifierRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)#todo 単語埋め込み層    in : x out : vocab_size x embedding_dim
        self.rnn =nn.LSTM(embedding_dim,hidden_dim)#todo LSTM層  in : vocab_size x embedding_dim  out : 
        self.linear = nn.Linear(hidden_dim,target_size)#todo 線形層  in : 系列x隠れ層　out : 系列x出力ラベル層

    def forward(self, x, x_len):  
        embed_x = self.embedding.forward(x)#todo xを単語埋め込み

        packed_x = pack_padded_sequence(embed_x, x_len, batch_first=True, enforce_sorted=False) #todo embed_xとx_lenを用いてPackedSequence型のデータに変換 
        _, (h, _) = self.rnn(packed_x)#todo LSTM層に入力
        output = self.liner(h)#ltodo 線形層に入力

        output = torch.squeeze(output,0)#todo squeeze()を使用し，outputの形状が[1, batch_size, target_size]となっているものを[batch_size, target_size]に変換

        #print(output.size()) #確認用(確認する場合はコメントアウトを解除)
        return output

class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.length = torch.from_numpy(np.array([len(i) for i in data]))#todo dataに含まれる各ID系列の長さのリスト
        self.label = label

        self.pad_data = pad_sequence(data, batch_first=True)#todo dataをpadding

    def __len__(self):
        return len(self.pad_data)

    def __getitem__(self, idx):
        return self.pad_data,self.length,self.label#todo idxに対応するpad_data, length, labelの3つを返り値とする



emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

vocab_size = max(word2id.values()) + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/train.txt'#todo 学習用ファイルパス
train_x, train_y = get_data(train_path)
valid_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/test.txt'#todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)


dataset = RnnDataset(train_x, train_y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = ClassifierRNN(vocab_size, emb_dim, hidden_dim, target_size)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, data_len, target in train_loader:
        # GPUへ
        data = data.to(device)
        data_len = data_len.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        pred = model.forward(data,data_len)#todo modelで予測
        loss = loss_fn(pred,target)#todo lossを計算

        loss.backward()
        optimizer.step()

        total_loss += loss#todo lossをtotal_lossに加算
        correct_num +=  (pred == target).sum().item()#todo 予測の正解数をカウント

    batch_loss = total_loss/len_train #バッチごとの平均ロス
    acc = correct_num/len_train#todo 正解率
    return batch_loss, acc

# validデータはそのままGPUへ
valid_x_len = []

for i in range(len(valid_x)):
  valid_x_len.append(valid_x[i].shape[0])#todo valid_xの各要素の長さのリスト
valid_x_len = torch.from_numpy(np.array(valid_x_len))

valid_pad_x = pad_sequence(valid_x, batch_first=True)#todo valid_xをpadding

# GPUへ
valid_x_len = valid_x_len.to(device)
valid_pad_x = valid_pad_x.to(device)
valid_y = valid_y.to(device)


def evaluation(model, data, data_len, target):
    model.eval()
    with torch.no_grad():
        pred = model.forward(data,data_len)#todo 予測
        loss = loss_fn(pred,target)#todo 損失を計算
        acc = torch.sum(pred_labels == target).item()/len(pred_labels)#todo 正解率を計算
    return loss.item(), acc

len_train = len(train_y)

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)
    valid_loss, valid_acc = evaluation(model, valid_pad_x, valid_x_len, valid_y)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

RuntimeError: ignored

In [ ]:
#長さのリストを返すテスト
testids_list = []
testword_list = ["UPDATE","Buy","effective","Macho"]


train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/train.txt'#todo 学習用ファイルパス
train_x, train_y = get_data(train_path)
#print(train_x[1].shape)

testl = []
for i in range(len(train_x)):
  #print(train_x[i].shape[0])
  testl.append(train_x[i].shape[0])
print(testl)
#リストの中のテンソルの大きさのリスト
#train_x_len = train_x[0:1:len(train_x)].shape

[7, 10, 11, 10, 9, 8, 16, 10, 12, 10, 7, 10, 11, 14, 16, 11, 9, 12, 10, 10, 13, 9, 9, 11, 11, 11, 10, 8, 17, 7, 9, 10, 10, 13, 8, 10, 11, 10, 9, 9, 9, 8, 8, 17, 17, 10, 9, 14, 10, 10, 16, 7, 17, 10, 8, 11, 11, 8, 9, 9, 15, 12, 11, 8, 11, 13, 10, 9, 14, 11, 13, 15, 10, 11, 11, 10, 13, 8, 10, 10, 16, 16, 11, 10, 10, 13, 11, 16, 2, 11, 10, 11, 12, 4, 18, 12, 9, 17, 12, 10, 6, 6, 12, 7, 10, 11, 10, 11, 9, 9, 10, 7, 9, 9, 13, 15, 8, 9, 10, 9, 10, 8, 13, 6, 10, 10, 14, 11, 5, 12, 9, 11, 13, 14, 10, 11, 14, 13, 9, 9, 14, 7, 13, 10, 13, 9, 14, 7, 8, 14, 11, 12, 8, 5, 11, 8, 8, 10, 13, 8, 6, 10, 11, 10, 10, 10, 10, 11, 10, 11, 14, 11, 10, 9, 13, 16, 9, 8, 7, 13, 11, 10, 13, 12, 9, 8, 12, 10, 10, 10, 13, 10, 12, 12, 11, 6, 14, 16, 11, 14, 15, 17, 9, 7, 18, 11, 11, 10, 10, 9, 4, 9, 8, 11, 10, 8, 9, 12, 10, 10, 11, 15, 9, 9, 12, 17, 16, 12, 10, 13, 8, 15, 9, 14, 9, 8, 11, 14, 8, 7, 10, 10, 10, 6, 10, 10, 10, 10, 14, 13, 7, 10, 11, 13, 9, 10, 12, 10, 11, 11, 7, 7, 8, 15, 10, 10, 14, 12, 14, 9, 14, 

In [ ]:
#LSTM
import torch
import torch.nn as nn

a = torch.rand(1, 4, 5)
print(a) #1x4x5
# 各系列の埋め込み次元数を5
# LSTM層の隠れ層のサイズは6
lstm = nn.LSTM(5, 6, batch_first=True, bidirectional=True) #
out, hc = lstm(a)#in 1x4x5
print("out:",out,out.shape)#out 1x4x12   hcの２倍？
#h 隠れ層、c 記憶セル
print("h:",hc,hc[0].shape)#2x1x6
print("c:",hc,hc[1].shape)

tensor([[[0.7550, 0.3687, 0.9389, 0.9964, 0.4501],
         [0.0244, 0.6510, 0.7585, 0.0705, 0.4373],
         [0.8907, 0.5042, 0.5403, 0.7755, 0.4224],
         [0.2613, 0.6990, 0.5274, 0.4734, 0.8897]]])
out: tensor([[[-0.0662, -0.1263, -0.0734, -0.0650, -0.0375,  0.1004,  0.1599,
           0.0971,  0.0254, -0.1665, -0.1352, -0.4217],
         [-0.1025, -0.1702, -0.1452, -0.1753, -0.0298,  0.1419,  0.1446,
           0.0552,  0.0867, -0.1274, -0.0699, -0.3110],
         [-0.0710, -0.1601, -0.1646, -0.1785, -0.0546,  0.2144,  0.1337,
           0.0626,  0.0884, -0.1193, -0.1300, -0.3885],
         [-0.0922, -0.2100, -0.2337, -0.2355,  0.0256,  0.2461,  0.0742,
           0.0424,  0.0627,  0.0114, -0.0836, -0.2490]]],
       grad_fn=<TransposeBackward0>) torch.Size([1, 4, 12])
h: (tensor([[[-0.0922, -0.2100, -0.2337, -0.2355,  0.0256,  0.2461]],

        [[ 0.1599,  0.0971,  0.0254, -0.1665, -0.1352, -0.4217]]],
       grad_fn=<StackBackward>), tensor([[[-0.1381, -0.4229, -0.3595, -0.

In [ ]:
# 次元を削減 squeeze()
a = torch.zeros(2,1,3,1)
print(a)
b = torch.squeeze(a)
print(b)
print(b.size())
c = torch.squeeze(a, 1).size() # dim = 1のみsqueeze
torch.Size([2, 3, 1])


tensor([[[[0.],
          [0.],
          [0.]]],


        [[[0.],
          [0.],
          [0.]]]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
torch.Size([2, 3])


torch.Size([2, 3, 1])

# 8章
8章 p353 Attention付きseq2seqを実行し，正解率を確認する

In [ ]:
import numpy as np

T,H = 5,4
hs = np.random.randn(T,H)#エンコーダーの出力
a = np.array([0.8,0.1,0.03,0.05,0.02])#各単語の重み
print(hs.shape)

ar = a.reshape(5,1).repeat(4,axis = 1)
#ab = a.reshape(5,1).repeat(4,axis = 0)
print(ar.shape)
#print(ab.shape)
t = hs*ar#畳み込み
print(t)
print(t.shape)

c = np.sum(t,axis = 0)#0軸めが消えるように和を求める
print(c)
print(c.shape)

(5, 4)
(5, 4)
[[ 0.66025553 -0.15136897 -0.07303481 -0.26322021]
 [-0.06001759  0.04182982  0.14125168  0.01855966]
 [ 0.00531913  0.06924237 -0.00648048  0.00281067]
 [ 0.03620831 -0.04146264  0.01852648  0.04539503]
 [-0.01356975  0.02697962  0.01629133  0.0253113 ]]
(5, 4)
[ 0.62819565 -0.0547798   0.09655422 -0.17114355]
(4,)


In [ ]:
N,T,H = 10,5,4
hs = np.random.randn(N,T,H)
a = np.random.randn(N,T)
ar = a.reshape(N,T,1)
#print(hs)
t = hs*ar
print(t.shape)
c = np.sum(t,axis=1)
print(c.shape)
#print(c)

(10, 5, 4)
(10, 4)


In [ ]:
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N,T,H = 10,5,4
hs = np.random.randn(N,T,H)#10x4
h = np.random.randn(N,H)#10x4
hr = h.reshape(N,1,H).repeat(T,axis = 1)#10x1x4を5回y方向にrepeatして10x5x4

t = hs*hr
print(t.shape)

s = np.sum(t,axis = 2)
print(s.shape)

softmax = Softmax()
a = softmax.forward(s)
print(s.shape)

(10, 5, 4)
(10, 5)
(10, 5)


In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.time_layers import *
from ch07.seq2seq import Encoder, Seq2seq
from ch08.attention_layer import TimeAttention


class AttentionEncoder(Encoder):
    def forward(self, xs):
        xs = self.embed.forward(xs)
        hs = self.lstm.forward(xs)
        return hs

    def backward(self, dhs):
        dout = self.lstm.backward(dhs)
        dout = self.embed.backward(dout)
        return dout


class AttentionDecoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        lstm_b = np.zeros(4 * H).astype('f')
        affine_W = (rn(2*H, V) / np.sqrt(2*H)).astype('f')
        affine_b = np.zeros(V).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
        self.attention = TimeAttention()
        self.affine = TimeAffine(affine_W, affine_b)
        layers = [self.embed, self.lstm, self.attention, self.affine]

        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, xs, enc_hs):
        h = enc_hs[:,-1]
        self.lstm.set_state(h)

        out = self.embed.forward(xs)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)

        return score

    def backward(self, dscore):
        dout = self.affine.backward(dscore)
        N, T, H2 = dout.shape
        H = H2 // 2

        dc, ddec_hs0 = dout[:,:,:H], dout[:,:,H:]
        denc_hs, ddec_hs1 = self.attention.backward(dc)
        ddec_hs = ddec_hs0 + ddec_hs1
        dout = self.lstm.backward(ddec_hs)
        dh = self.lstm.dh
        denc_hs[:, -1] += dh
        self.embed.backward(dout)

        return denc_hs

    def generate(self, enc_hs, start_id, sample_size):
        sampled = []
        sample_id = start_id
        h = enc_hs[:, -1]
        self.lstm.set_state(h)

        for _ in range(sample_size):
            x = np.array([sample_id]).reshape((1, 1))

            out = self.embed.forward(x)
            dec_hs = self.lstm.forward(out)
            c = self.attention.forward(enc_hs, dec_hs)
            out = np.concatenate((c, dec_hs), axis=2)
            score = self.affine.forward(out)

            sample_id = np.argmax(score.flatten())
            sampled.append(sample_id)

        return sampled


class AttentionSeq2seq(Seq2seq):
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        args = vocab_size, wordvec_size, hidden_size
        self.encoder = AttentionEncoder(*args)
        self.decoder = AttentionDecoder(*args)
        self.softmax = TimeSoftmaxWithLoss()

        self.params = self.encoder.params + self.decoder.params
        self.grads = self.encoder.grads + self.decoder.grads

In [ ]:
#Attation付きseq2seqの学習
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch07.seq2seq import Seq2seq
from ch07.peeky_seq2seq import PeekySeq2seq

#読み込み
(x_train,t_train),(x_test,t_test) = sequence.load_data('date.txt')
char_to_id,id_to_char = sequence.load_data('date.txt')
char_to_id,id_to_char = sequence.get_vocab()

#入力文を反転
x_train,x_test = x_train[:,::-1],x_test[:,::-1]

#ハイパーパラメータの設定
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5.0

model = AttentionSeq2seq(vocab_size,wordvec_size,hidden_size)
optimizer = Adam()
trainer = Trainer(model,optimizer)

acc_list = []
for epoch in range(max_epoch):
  trainer.fit(x_train,t_train,max_epoch = 1,batch_size = batch_size,max_grad = max_grad)

  correct_num = 0
  for i in range(len(x_test)):
    question,correct = x_test[[i]],t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model,question,correct,id_to_char,verbose,is_reverse = True)

    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc*100))

model.save_params()

ストリーミング出力は最後の 5000 行に切り捨てられました。
T 2013-10-28
☑ 2013-10-28
---
val acc 0.180%
Q sunday, november 6, 2016     
T 2016-11-06
☑ 2016-11-06
---
val acc 0.200%
val acc 0.220%
val acc 0.240%
val acc 0.260%
val acc 0.280%
val acc 0.300%
val acc 0.320%
val acc 0.340%
val acc 0.360%
val acc 0.380%
val acc 0.400%
val acc 0.420%
val acc 0.440%
val acc 0.460%
val acc 0.480%
val acc 0.500%
val acc 0.520%
val acc 0.540%
val acc 0.560%
val acc 0.580%
val acc 0.600%
val acc 0.620%
val acc 0.640%
val acc 0.660%
val acc 0.680%
val acc 0.700%
val acc 0.720%
val acc 0.740%
val acc 0.760%
val acc 0.780%
val acc 0.800%
val acc 0.820%
val acc 0.840%
val acc 0.860%
val acc 0.880%
val acc 0.900%
val acc 0.920%
val acc 0.940%
val acc 0.960%
val acc 0.980%
val acc 1.000%
val acc 1.020%
val acc 1.040%
val acc 1.060%
val acc 1.080%
val acc 1.100%
val acc 1.120%
val acc 1.140%
val acc 1.160%
val acc 1.180%
val acc 1.200%
val acc 1.220%
val acc 1.240%
val acc 1.260%
val acc 1.280%
val acc 1.300%
val acc 1.320%
val ac

# 感想
８章では、seq2seqの改善を行った。この時問題点として上がっていたものは、今回100本ノックの方で使用したものと目的は一致していた。それぞれ手段は違うけど解決したいことは両方「入力の単語数が違うのに出力されるベクトルが固定長だと情報の正しさが低くなる」であった。ゼロから作るの方は、LSTMレイヤの隠れ状態ベクトルを全て利用することで改善としていた。100本ノックの83番の方は、入力の型をTensorからPackedSequence型に変更することで改善した。<br>
ゼロから作るの方の結果を見てみる。非常に高い精度で学習できているかつ、精度が高くなるのが今までのモデルに比べて早いことがわかった。重要な単語に注目することで学習速度が短縮できているのだと考えられる。<br>
100本ノックの方では、PackedSequence型を使ってLSTMで学習する方法だった。メモリエラー？が解決できず実行できなかったので結果はどうなるかわからなかった。